# Goal
- Train GPT2 on wiki text

## Steps
- Read, download data
- Train tokenizer
- Prepare sliding window data loader
- Use GPT2 model
- Use train/test loop

### Read, download data

In [2]:
from datasets import load_dataset

train_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train")
val_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="validation")
test_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")

train_dataset

Dataset({
    features: ['text'],
    num_rows: 36718
})

In [3]:
type(train_dataset['text'][1])

str

### Train tokenizer

In [4]:
import tokenizers
import transformers
import tiktoken

tokenizer = tokenizers.Tokenizer(tokenizers.models.BPE())
tokenizer.pre_tokenizer = tokenizers.pre_tokenizers.ByteLevel(add_prefix_space=False)

trainer = tokenizers.trainers.BpeTrainer(vocab_size=25000, special_tokens=["<|endoftext|>", "<pad>"])
tokenizer.train_from_iterator(train_dataset["text"], trainer=trainer)
tokenizer.post_processor = tokenizers.processors.ByteLevel(trim_offsets=False)

tokenizer.save("../data/tokenizer.json")
tokenizer.decoder = tokenizers.decoders.ByteLevel()

wrapped_tokenizer = transformers.PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<|endoftext|>",
    eos_token="<|endoftext|>",
    padding_side="left",
    pad_token="<pad>",
    max_len=128
)


tokenizer.encode("Hello my name is Ajay").tokens


['H', 'ello', 'Ġmy', 'Ġname', 'Ġis', 'ĠAj', 'ay']

In [5]:
wrapped_tokenizer("<|endoftext|>")['input_ids']

[0]

In [6]:
wrapped_tokenizer(["Hello my name is Ajay"])['input_ids']

[[41, 14980, 1669, 1222, 302, 18604, 289]]

### Prepare sliding window data loader

In [7]:
import torch 

def slide_window(text_batch):
    text_batch['input_words'] = []
    text_batch['output_words'] = []
    text_batch['input_ids_raw'] = []
    text_batch['output_ids_raw'] = []
    text_batch['tokens'] = []
    text_batch['input_text'] = []
    text_batch['output_text'] = []

    text_batch['input_ids'] = []
    text_batch['output_ids'] = []

    for text in text_batch['text']:
        tokens = wrapped_tokenizer.tokenize(text)
        text_batch['tokens'].append(tokens)

        # Create input and output tokens for sliding window
        input_tokens = tokens[:-1]
        output_tokens = tokens[1:]

        input_ids = wrapped_tokenizer.convert_tokens_to_ids(input_tokens)
        output_ids = wrapped_tokenizer.convert_tokens_to_ids(output_tokens)

        text_batch['input_words'].append(input_tokens)
        text_batch['output_words'].append(output_tokens)

        text_batch['input_ids_raw'].append(input_ids)
        text_batch['output_ids_raw'].append(output_ids)
    
        input_text = wrapped_tokenizer.convert_tokens_to_string(input_tokens)
        output_text = wrapped_tokenizer.convert_tokens_to_string(output_tokens)
        
        text_batch['input_text'].append(input_text)
        text_batch['output_text'].append(output_text)

        input_ids = wrapped_tokenizer(input_text, return_tensors="pt", padding="max_length", truncation=True, max_length=200).input_ids[0]
        output_ids = wrapped_tokenizer(output_text, return_tensors="pt", padding="max_length", truncation=True, max_length=200).input_ids[0]

        text_batch['input_ids'].append(input_ids)
        text_batch['output_ids'].append(output_ids)


    return text_batch 

tokenized_train_dataset = train_dataset.map(slide_window, batched=True)
tokenized_val_dataset = val_dataset.map(slide_window, batched=True)
tokenized_test_dataset = test_dataset.map(slide_window, batched=True)

tokenized_train_dataset


Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (165 > 128). Running this sequence through the model will result in indexing errors


Dataset({
    features: ['text', 'input_words', 'output_words', 'input_ids_raw', 'output_ids_raw', 'tokens', 'input_text', 'output_text', 'input_ids', 'output_ids'],
    num_rows: 36718
})

In [8]:
tokenized_train_dataset[1]

{'text': ' = Valkyria Chronicles III = \n',
 'input_words': ['Ġ=', 'ĠValkyria', 'ĠChronicles', 'ĠIII', 'Ġ='],
 'output_words': ['ĠValkyria', 'ĠChronicles', 'ĠIII', 'Ġ=', 'ĠĊ'],
 'input_ids_raw': [239, 8577, 9442, 2988, 239],
 'output_ids_raw': [8577, 9442, 2988, 239, 253],
 'tokens': ['Ġ=', 'ĠValkyria', 'ĠChronicles', 'ĠIII', 'Ġ=', 'ĠĊ'],
 'input_text': ' = Valkyria Chronicles III =',
 'output_text': ' Valkyria Chronicles III = \n',
 'input_ids': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,


In [9]:
import torch 
from datasets import Dataset as HFDataset
from torch.utils.data import Dataset

class HuggingFaceDataset(Dataset):
    """
    Wraps a Hugging Face Dataset to be used with a PyTorch DataLoader.

    Assumes the Hugging Face dataset has 'input' and 'target' columns.
    """

    def __init__(self, hf_dataset: HFDataset):
        self.hf_dataset = hf_dataset

    def __len__(self):
        return len(self.hf_dataset)

    def __getitem__(self, idx):
        item = self.hf_dataset[idx]
        return item['input_ids'], item['output_ids']

def collate_fn(batch):
    input_ids = [item[0] for item in batch]
    output_ids = [item[1] for item in batch]
    input_ids_list = torch.tensor(input_ids)
    output_ids_list = torch.tensor(output_ids)
    return input_ids_list, output_ids_list

batch_size = 200
train_torch_dataset = HuggingFaceDataset(tokenized_train_dataset)
val_torch_dataset = HuggingFaceDataset(tokenized_val_dataset)
test_torch_dataset = HuggingFaceDataset(tokenized_test_dataset)

train_torch_dataloader = torch.utils.data.DataLoader(
    train_torch_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn
)
val_torch_dataloader = torch.utils.data.DataLoader(
    val_torch_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn
)
test_torch_dataloader = torch.utils.data.DataLoader(
    test_torch_dataset,
    batch_size=batch_size,
    shuffle=False,
    collate_fn=collate_fn
)

train_torch_dataloader

In [10]:
batch = next(iter(train_torch_dataloader)) # (input_ids, output_ids)
input_ids, output_ids = batch
input_ids.shape, output_ids.shape

(torch.Size([200, 200]), torch.Size([200, 200]))

### Use GPT2 model

In [11]:
from models import GPT2

num_train_batches = tokenized_train_dataset.num_rows // batch_size

config = {
        "emb_dim": 768,
        "heads": 12,
        "layers": 12,
        "vocab_size": 50257,
        "context_length": 128,
        "device": torch.device("cuda"),
        "drop_out": 0.1,
        "train_test_split": 0.8,
        "num_epochs": 5,
        "model_path": "../model_files/gpt2.pth",
        "num_train_batches" : num_train_batches
    }

gpt2 = GPT2(config)
gpt2.to(config['device'])
gpt2

GPT2(
  (token_embedding): Embedding(50257, 768)
  (position_embedding): Embedding(128, 768)
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (layer_norm1): LayerNorm()
      (layer_norm2): LayerNorm()
      (self_attention_block): MultiHeadAttention(
        (W_Q): Linear(in_features=768, out_features=768, bias=True)
        (W_K): Linear(in_features=768, out_features=768, bias=True)
        (W_V): Linear(in_features=768, out_features=768, bias=True)
        (out_project): Linear(in_features=768, out_features=768, bias=True)
      )
      (feed_forward): FeedForward(
        (feed_forward): Sequential(
          (0): Linear(in_features=768, out_features=3072, bias=True)
          (1): GELU(approximate='none')
          (2): Linear(in_features=3072, out_features=768, bias=True)
        )
      )
    )
    (1): TransformerBlock(
      (layer_norm1): LayerNorm()
      (layer_norm2): LayerNorm()
      (self_attention_block): MultiHeadAttention(
        (W_Q): Linear(in

### Use train/test loop

In [12]:
from utils import train

train(gpt2, train_torch_dataloader, val_torch_dataloader, config)
torch.save(gpt2.state_dict(), config["model_path"]) # Save the model

/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1422: indexSelectLargeIndex: block: [55,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1422: indexSelectLargeIndex: block: [55,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1422: indexSelectLargeIndex: block: [55,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1422: indexSelectLargeIndex: block: [55,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1422: indexSelectLargeIndex: block: [55,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1422: indexSelectLargeIndex: block: [55,0,0], thread: [5,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
/pytorch/aten/src/ATen/native/cuda/Indexing.cu:1422: index

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [ ]:
# load GPT2 from config.model_path
import os 

if os.path.exists(config['model_path']):
    gpt2.load_state_dict(torch.load(config['model_path']))
    print("model loaded")



### Generate text

In [ ]:
dir(wrapped_tokenizer)

In [ ]:
tokenized = wrapped_tokenizer("Hello my name is", truncation=True, max_length=100, padding="max_length", return_tensors="pt")

attention_mask = tokenized['attention_mask'].to(config["device"])
input_ids = tokenized['input_ids'].to(config["device"])

print(attention_mask)
print(input_ids)

prediction = gpt2(input_ids)
next_token = prediction.argmax(dim=-1)

print(prediction.shape)

In [ ]:

def generate_text(starting_text, model, tokenizer, config):
    input_encoding = tokenizer(starting_text, return_tensors="pt")
    device = config["device"]
    output_text = ""
    output_tokens = min(100, tokenizer.model_max_length, len(input_encoding['input_ids'][0]))
    for i in range(output_tokens):
        text = starting_text + output_text
        input_encoding = tokenizer(text, truncation=True, max_length=100, padding="max_length", return_tensors="pt")
        input_ids = input_encoding['input_ids'].to(device)
        input_attention_mask = input_encoding['attention_mask'].to(device)[0]
        next_token_logits = model(input_ids)[:,-1,:]
        

        next_token = next_token_logits.argmax(dim=-1)
        # next_token_probs = torch.softmax(next_token_logits, dim=-1)
        # next_token = torch.multinomial(next_token_probs, num_samples=1)
        print(next_token)
        next_text = tokenizer.decode(next_token[0], skip_special_tokens=True)
        output_text += next_text
    print(f"{starting_text} -> {output_text}")

generate_text("The capital of United States of America", gpt2, wrapped_tokenizer, config)


In [ ]:
wrapped_tokenizer.encode("Who is the president of the United States?")